In [1]:
import pandas as pd
import pyspark.pandas as ps
import requests
import json

from sqlalchemy import create_engine

from pyspark.sql.functions import pandas_udf, PandasUDFType, max, col, countDistinct, when, rank, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType
from pyspark.sql.window import Window

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from imblearn.under_sampling import ClusterCentroids



# from backend_spark.doris_common.doris_client import DorisClient
from functools import reduce
from pyspark.sql import DataFrame
from typing import Optional

In [7]:
#####################################################################################
#######################################解析SQL########################################
#####################################################################################

In [22]:
# ## 这里使用doris数据库连接
# user ="root"
# host = "10.52.199.81"
# password = "Nexchip%40123"
# db = "etl"
# port = 9030

# engine = create_engine("mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(user = user,
#                                                                                     password = password,
#                                                                                     host = host,
#                                                                                     port = port,
#                                                                                     db = db))

# df1_pandas = pd.read_sql_query("SELECT * FROM etl.DWD_POC_CASE_FD_UVA_DATA_CASE1_PROCESSED1", engine)   

In [2]:
df1_pandas = pd.read_csv("DWD_POC_CASE_FD_UVA_DATA_CASE1_PROCESSED1.csv")
df1_pandas

,WAFER_ID,TOOL_ID,RUN_ID,EQP_NAME,PRODUCT_ID,PRODG1,TOOL_NAME,LOT_ID,RECIPE_NAME,OPER_NO,START_TIME,parametric_name,CASE_INFO,STATUS,STATISTIC_RESULT,label
0,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_5_CHF3#WINDOW_1#SUM,2023-06-16,NORMAL,123907.6000,0
1,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_4_CH2F2#WINDOW_1#SUM,2023-06-16,NORMAL,0.0000,0
2,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_6_CF4#WINDOW_1#SUM,2023-06-16,NORMAL,0.0000,0
3,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,ESC_CURRENT#AOTU_STEP_2#MEAN,2023-06-16,NORMAL,62.0000,0
4,NGE186-06,11341,149769,EKT72,AFKN2J01N.0U01,L11CD02A,EKT72_PM1,NGE186.000,NEW-DRM/P1/110NM/PFKN0S0D1F1A,1F.EEK10,2023-06-16 02:12:04,PROCESS_GAS_2_C4F8#AOTU_STEP_2#MEAN,2023-06-16,NORMAL,0.0000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550906,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,ESC_VOLTAGE#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,2503.0000,0
550907,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_11_N2#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,0.0000,0
550908,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_4_CH2F2#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,0.0000,0
550909,NGG239-19,11341,153513,EKT72,AMKNS301N.0B01,L11CB14A,EKT72_PM1,NGG239.000,NEW-DRM/P1/110NM/PFKN9F0D1F1A,1F.EEK10,2023-06-30 12:01:39,PROCESS_GAS_7_AR#AOTU_STEP_2#MEAN,2023-06-30,NORMAL,199.9009,0


In [3]:
df1 = ps.from_pandas(df1_pandas).to_spark()

/usr/local/spark/python/pyspark/pandas/internal.py:1573: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  fields = [
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [4]:
df1.count()

550911

In [6]:
bad_wafer_num = df1.filter("label == 1").select('WAFER_ID').distinct().count()
good_wafer_num = df1.filter("label == 0").select('WAFER_ID').distinct().count()
print(bad_wafer_num)
print(good_wafer_num)

2685
3409


In [7]:
# df1.filter("label == 1").select('WAFER_ID').distinct().count()

In [7]:
############################################################################
##################################FDC数据预处理###############################
############################################################################
def _pre_process(df):
    """
    param df: 从数据库中读取出来的某个CASE数据
    return: 数据预处理，后面要根据实际情况统一添加
    """
    # 只选出会用到的列
    df = df.select('WAFER_ID', 'TOOL_ID', 'RUN_ID', 'EQP_NAME', 'PRODUCT_ID', 'PRODG1', 
                'TOOL_NAME', 'OPER_NO', 'parametric_name', 'STATISTIC_RESULT', 'label')
    # 剔除NA值
    df = df.filter(col('STATISTIC_RESULT').isNotNull())
    # 按照所有的行进行去重
    df1 = df.dropDuplicates()
    # 选最新的RUN
    df2 = df1.groupBy('WAFER_ID', 'OPER_NO', 'TOOL_ID').agg(max('RUN_ID').alias('RUN_ID'))
    df_run = df1.join(df2.dropDuplicates(subset=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID']),
                                on=['WAFER_ID', 'OPER_NO', 'TOOL_ID', 'RUN_ID'], how='inner')
    return df_run


def commonality_analysis(df_run, grpby_list):
    """
    param df_run: 数据预处理后的数据
    return: 共性分析后的结果， 返回bad wafer前五的组合
    """
    grps = (df_run.groupBy(grpby_list)
                        .agg(countDistinct('WAFER_ID').alias('wafer_count'),
              countDistinct('WAFER_ID', when(df_run['label'] == 0, 1)).alias('good_num'),
              countDistinct('WAFER_ID', when(df_run['label'] == 1, 1)).alias('bad_num'))
              .orderBy('bad_num', ascending=False))

    # 单站点+单腔室的情况
    if grps.count() == 1:
        return grps
    else: 
        grps = grps.filter(grps['bad_num'] > 0)
        window_sep = Window().orderBy(col("bad_num").desc())
        ranked_df = grps.withColumn("rank", rank().over(window_sep))
        grpss = ranked_df.filter(col("rank") <= 5).drop("rank")
    return grpss

In [8]:
df_run = _pre_process(df1)
print(df_run.count())

550896


In [9]:
grpby_list = ['PRODG1', 'OPER_NO', 'TOOL_NAME']
common_res = commonality_analysis(df_run, grpby_list)
common_res.show()

+--------+--------+---------+-----------+--------+-------+
|  PRODG1| OPER_NO|TOOL_NAME|wafer_count|good_num|bad_num|
+--------+--------+---------+-----------+--------+-------+
|L11TG07A|1F.EEK10|EKT72_PM1|       1000|       0|   1000|
|L11CD02A|1F.EEK10|EKT72_PM1|        959|     234|    725|
|L11TG05A|1F.EEK10|EKT72_PM1|        299|       0|    299|
|L11EG07A|1F.EEK10|EKT72_PM1|        224|       0|    224|
|L15KD03A|1F.EEK10|EKT72_PM1|        101|       0|    101|
+--------+--------+---------+-----------+--------+-------+



In [42]:
############################################################################
#################################获取有good和bad的数据#########################
############################################################################
def get_data_list_big_sample(common_res, grpby_list):
    """
    param common_res: 共性分析后的结果, 按照good_num >= 3 AND bad_num >= 3筛选出组合
    return: 对应组合的字典形式
    """
    good_bad_grps = common_res.filter("good_num >= 3 AND bad_num >= 3")
    
    if grpby_list == ['PRODG1', 'OPER_NO', 'TOOL_NAME']:
        data_list = good_bad_grps['PRODG1', 'OPER_NO', 'TOOL_NAME'].collect()
        
    elif grpby_list == ['OPER_NO', 'TOOL_NAME']:
        data_list = good_bad_grps['OPER_NO', 'TOOL_NAME'].collect()
        
    else:
        raise ValueError("分组字段只能是PRODG1+OPER_NO+TOOL_NAME或OPER_NO+TOOL_NAME, 请检查！")
        
    data_dict_list = [row.asDict() for row in data_list]
    return data_dict_list




def get_train_data_big_sample(df_run, data_dict_list):
    """
    param df_run: 数据预处理后的数据
    param data_dict: 筛选后的字典结果(get_data_list_big_sample)
    return: 从原始数据中过滤出大样本组合的数据
    """
    if len(data_dict_list[0]) == 3:
        prod, oper, tool = data_dict_list[0]['PRODG1'], data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
        df_s = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' AND TOOL_NAME == '{}'".format(prod, oper, tool))
        for i in range(1, len(data_dict_list)):
            prod, oper, tool = data_dict_list[i]['PRODG1'], data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
            df_m = df_run.filter("PRODG1 == '{}' AND OPER_NO == '{}' and TOOL_NAME == '{}'".format(prod, oper, tool))
            df_s = df_s.union(df_m)

    elif len(data_dict_list[0]) == 2:
        oper, tool = data_dict_list[0]['OPER_NO'], data_dict_list[0]['TOOL_NAME']
        df_s = df_run.filter("OPER_NO == '{}' AND TOOL_NAME == '{}'".format(oper, tool))
        for i in range(1, len(data_dict_list)):
            oper, tool = data_dict_list[i]['OPER_NO'], data_dict_list[i]['TOOL_NAME']
            df_m = df_run.filter("OPER_NO == '{}' and TOOL_NAME == '{}'".format(oper, tool))
            df_s = df_s.union(df_m)
    else:
        raise ValueError("数据库中暂时找不到数据, 请检查！")
    return df_s

In [40]:
data_dict_list_bs = get_data_list_big_sample(common_res, ['PRODG1', 'OPER_NO', 'TOOL_NAME'])
data_dict_list_bs

[{'PRODG1': 'L11CD02A', 'OPER_NO': '1F.EEK10', 'TOOL_NAME': 'EKT72_PM1'}]

In [43]:
df_run_bs = get_train_data_big_sample(df_run=df_run, data_dict_list=data_dict_list_bs)
print(df_run_bs.count())

57540


In [45]:
############################################################################
#########################获取传入的整个数据中的所有bad_wafer个数####################
############################################################################
def get_all_bad_wafer_num(df):
    """
    param df: 筛选后的数据
    return: 数据中所有bad_wafer的数量
    """
    return df.filter("label == 1").select('WAFER_ID').distinct().count()

In [46]:
bad_wafer_num_big_sample = get_all_bad_wafer_num(df_run_bs)
bad_wafer_num_big_sample

725

In [76]:
############################################################################
###########################对有good和bad的数据，建模############################
############################################################################
def fit_rf_big_sample(df, by):
    """
    param df: 大样本组合的数据
    param by: 分组字段
    return: RandomForest建模后的结果
    """
    schema_all = StructType([StructField("PRODG1", StringType(), True),
                             StructField("OPER_NO", StringType(), True),
                             StructField("TOOL_NAME", StringType(), True),
                             StructField("bad_wafer", IntegerType(), True),
                             StructField("roc_auc_score", FloatType(), True),
                             StructField("features", StringType(), True),
                             StructField("importance", FloatType(), True)]) 
    
    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_model_result(df_run):
        if len(by) == 3: 
            df_pivot = df_run.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                         columns=['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name'],
                                                         values=['STATISTIC_RESULT'])
        else:
            df_pivot = df_run.dropna(axis=0).pivot_table(index=['WAFER_ID', 'label'], 
                                                         columns=['OPER_NO', 'TOOL_NAME', 'parametric_name'],
                                                         values=['STATISTIC_RESULT'])
        
        df_pivot.columns = df_pivot.columns.map('#'.join)
        df_pivot = df_pivot.fillna(df_pivot.mean()).reset_index(drop=False)

        # 定义自变量和因变量
        X_train = df_pivot[df_pivot.columns.difference(['WAFER_ID', 'label']).tolist()]
        y_train = df_pivot[['label']]

        z_ratio = y_train.value_counts(normalize = True)
        good_ratio = z_ratio[0]
        bad_ratio = z_ratio[1]
        if abs(good_ratio - bad_ratio) > 0.7:
            undersampler = ClusterCentroids(random_state=101)
            X_train, y_train = undersampler.fit_resample(X_train, y_train)

        # 网格搜索
        pipe = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='constant', fill_value=-999)),
            ('scaler', StandardScaler()),
            ('model', RandomForestClassifier())])
        param_grid = {'model__n_estimators': [*range(50, 100, 10)],
                                'model__max_depth': [*range(10, 50, 10)]}
        grid = GridSearchCV(estimator=pipe, scoring='roc_auc', param_grid=param_grid, cv=3, n_jobs=-1)
        grid.fit(X_train.values, y_train.values.ravel())
        roc_auc_score_ = grid.best_score_

        # 特征重要度、结果汇总
        small_importance_res = pd.DataFrame({
            'features': X_train.columns,
            'importance': grid.best_estimator_.steps[2][1].feature_importances_}).sort_values(by='importance', ascending=False)
        
        if len(by) == 3: 
            small_sample_res = pd.DataFrame({'PRODG1': df_run['PRODG1'].unique(),
                                            'OPER_NO': df_run['OPER_NO'].unique(),
                                            'TOOL_NAME': df_run['TOOL_NAME'].unique(),
                                            'bad_wafer': sum(df_pivot['label']),
                                            'roc_auc_score': roc_auc_score_})
        else:
            PRODG1 = 'grplen2'
            small_sample_res = pd.DataFrame({'PRODG1': PRODG1,
                                            'OPER_NO': df_run['OPER_NO'].unique(),
                                            'TOOL_NAME': df_run['TOOL_NAME'].unique(),
                                            'bad_wafer': sum(df_pivot['label']),
                                            'roc_auc_score': roc_auc_score_})
            
        return pd.concat([small_importance_res, small_sample_res])
    return df.groupby(by).apply(get_model_result)

In [92]:
res = fit_rf_big_sample(df=df_run_bs, by=['OPER_NO', 'TOOL_NAME'])
res.show()

+------+-------+---------+---------+-------------+--------------------+-----------+
|PRODG1|OPER_NO|TOOL_NAME|bad_wafer|roc_auc_score|            features| importance|
+------+-------+---------+---------+-------------+--------------------+-----------+
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.14183468|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.102675036|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.098108254|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.06699185|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.06535871|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.05160663|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...| 0.04442758|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.032958783|
|  null|   null|     null|     null|         null|STATISTIC_RESULT#...|0.030

In [95]:
res.toPandas()[['features']].iloc[0].values

array(['STATISTIC_RESULT#1F.EEK10#EKT72_PM1#LO_RF_VPP#STEP2_MEANDIFF#MEAN'],
      dtype=object)

In [85]:
# res.filter(col('PRODG1').isNotNull()).show()

In [117]:
#####################################################################################
#########################对有good和bad的建模后的结果进行整合###############################
#####################################################################################
def split_score_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: roc_auc分数结果
    """
    schema_all = StructType([ StructField("PRODG1", StringType(), True),
                              StructField("OPER_NO", StringType(), True),
                              StructField("TOOL_NAME", StringType(), True),
                              StructField("bad_wafer", IntegerType(), True),
                              StructField("roc_auc_score", FloatType(), True) ]) 
     
    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(model_results):
        sample_res = model_results[['PRODG1', 'OPER_NO', 'TOOL_NAME', 'bad_wafer', 'roc_auc_score']].dropna(axis=0)
        sample_res = sample_res[sample_res['roc_auc_score'] > 0.7]
        return sample_res
    return df.groupby(by).apply(get_result)



def split_calculate_features_big_sample(df, by):
    """
    param df: RandomForest建模后的结果
    param by: 分组字段
    return: features和importance结果
    """
    schema_all = StructType([ StructField("PRODG1", StringType(), True),
                              StructField("OPER_NO", StringType(), True),
                              StructField("TOOL_NAME", StringType(), True),
                              StructField("parametric_name", StringType(), True),
                              StructField("importance", FloatType(), True),
                              StructField("stats", StringType(), True)]) 
     
    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(model_results):
        # 先从随机森林的模型结果中取出包含features和importance的dataframe
        feature_importance_table = model_results[['features',  'importance']].dropna(axis=0)
        
        if len(by) == 3:
            # 对其进行split
            feature_importance_table['STATISTIC_RESULT'] = feature_importance_table['features'].apply(lambda x: x.split('#')[0])
            feature_importance_table['PRODG1'] = feature_importance_table['features'].apply(lambda x: x.split('#')[1])
            feature_importance_table['OPER_NO'] = feature_importance_table['features'].apply(lambda x: x.split('#')[2])
            feature_importance_table['TOOL_NAME'] = feature_importance_table['features'].apply(lambda x: x.split('#')[3])
            feature_importance_table['parametric_name'] = feature_importance_table['features'].apply(lambda x: x.split('#')[4])
            feature_importance_table['step'] = feature_importance_table['features'].apply(lambda x: x.split('#')[5])
            feature_importance_table['stats'] = feature_importance_table['features'].apply(lambda x: x.split('#')[6])
            feature_importance_res_split = feature_importance_table.drop(['features', 'STATISTIC_RESULT'], axis=1).reset_index(drop=True)
        else:
            feature_importance_table['STATISTIC_RESULT'] = feature_importance_table['features'].apply(lambda x: x.split('#')[0])
            feature_importance_table['OPER_NO'] = feature_importance_table['features'].apply(lambda x: x.split('#')[1])
            feature_importance_table['TOOL_NAME'] = feature_importance_table['features'].apply(lambda x: x.split('#')[2])
            feature_importance_table['parametric_name'] = feature_importance_table['features'].apply(lambda x: x.split('#')[3])
            feature_importance_table['step'] = feature_importance_table['features'].apply(lambda x: x.split('#')[4])
            feature_importance_table['stats'] = feature_importance_table['features'].apply(lambda x: x.split('#')[5])
            feature_importance_res_split = feature_importance_table.drop(['features', 'STATISTIC_RESULT'], axis=1).reset_index(drop=True)
            feature_importance_res_split = feature_importance_res_split.assign(PRODG1 = 'grplen2')

        # 去除importance为0的组合
        feature_importance_res_split_drop = feature_importance_res_split.loc[feature_importance_res_split['importance'] != 0].reset_index(drop=True)
        
        #取每一种组合结果的前60%
        feature_importance_res_split_nlargest = (feature_importance_res_split_drop.groupby(by = ['PRODG1', 'OPER_NO', 'TOOL_NAME'])
                                                .apply(lambda x:x.nlargest(int(x.shape[0]*0.6), 'importance'))
                                                .reset_index(drop=True))
        
        # 新增一列，含有参数的所有统计特征:feature_stats
        feature_stats = feature_importance_res_split_nlargest.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name','step'])['stats'].unique().reset_index()
        feature_stats['stats'] = [feature_stats['stats'].iloc[i].tolist() for i in range(len(feature_stats))]
        feature_stats['stats'] = feature_stats['stats'].apply(lambda x: "#".join(x))
        feature_stats = feature_stats.assign(parametric_name=lambda x: x['parametric_name']+str('#')+x['step']).drop('step', axis=1)

        #对同一种组合里的同一个参数进行求和:feature_importance_groupby
        feature_importance_groupby = (feature_importance_res_split_nlargest.groupby(['PRODG1', 'OPER_NO', 'TOOL_NAME', 
                                                                                     'parametric_name','step'])['importance'].sum().reset_index())
        feature_importance_groupby = feature_importance_groupby.assign(parametric_name=lambda x: x['parametric_name']+str('#')+x['step']).drop('step', axis=1)

        # feature_stats和feature_importance_groupby连接
        grpby_stats = pd.merge(feature_stats, feature_importance_groupby, on=['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name']).dropna().reset_index(drop=True)
        return grpby_stats
    
    return df.groupby(by).apply(get_result)



def get_finall_results_big_sample(s_res, f_res, bad_wafer_num, grpby_list):
    """
    param s_res: roc_auc分数结果
    param f_res: features和importance结果
    param bad_wafer_num: 数据中所有bad_wafer的数量
    return: 最后的建模结果
    """
    # feature_importance_groupby和sample_res连接
    df_merge = s_res.join(f_res, on=['PRODG1', 'OPER_NO', 'TOOL_NAME'], how='left')
    df_merge = (df_merge.withColumn('bad_ratio', col('bad_wafer')/bad_wafer_num)
                        .withColumn('weight', col('roc_auc_score')*col('importance')*col('bad_ratio'))
                        .orderBy('weight', ascending=False))
    
    if len(grpby_list) == 3:
        df_merge = df_merge.select(['PRODG1', 'OPER_NO', 'TOOL_NAME', 'parametric_name', 'weight', 'stats'])
    else:
        df_merge = df_merge.select(['OPER_NO', 'TOOL_NAME', 'parametric_name', 'weight', 'stats'])
    return df_merge

In [93]:
s_res = split_score_big_sample(df=res, by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
s_res.show()

+-------+--------+---------+---------+-------------+
| PRODG1| OPER_NO|TOOL_NAME|bad_wafer|roc_auc_score|
+-------+--------+---------+---------+-------------+
|grplen2|1F.EEK10|EKT72_PM1|      725|   0.86156595|
+-------+--------+---------+---------+-------------+



In [101]:
f_res = split_calculate_features_big_sample(df=res, by=['OPER_NO', 'TOOL_NAME'])
f_res.show()

+-------+--------+---------+--------------------+-----------+----------+
| PRODG1| OPER_NO|TOOL_NAME|     parametric_name| importance|     stats|
+-------+--------+---------+--------------------+-----------+----------+
|grplen2|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...|0.028123723|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#...|0.020992685|     RANGE|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_GAS_PRESSURE...|0.026407281|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|EDGE_HE_FLOW#AOTU...|0.032736212|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|FLOWSPLITCENTER#A...|  0.0288561|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|FLOWSPLITEDGE#AOT...| 0.02685907|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LOWERBRINETEMP#AO...|0.021333966|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LOWER_TEMPERATURE...|0.051086895|      MEAN|
|grplen2|1F.EEK10|EKT72_PM1|LO_C1_VAR_CAPACIT...| 0.04719861|     SLOPE|
|grplen2|1F.EEK10|EKT72_PM1|LO_C2_VAR_CAPACIT...|0.031333793|     RANGE|
|grplen2|1F.EEK10|EKT72_PM1|LO_RF_POWER#AOTU_...| 0

In [119]:
model_res_bs = get_finall_results_big_sample(s_res=s_res, f_res=f_res, 
                                             bad_wafer_num=bad_wafer_num_big_sample, 
                                             grpby_list=['OPER_NO', 'TOOL_NAME'])
model_res_bs.show()

+--------+---------+--------------------+--------------------+----------+
| OPER_NO|TOOL_NAME|     parametric_name|              weight|     stats|
+--------+---------+--------------------+--------------------+----------+
|1F.EEK10|EKT72_PM1|LO_RF_VPP#AOTU_ST...| 0.14697401225566864|MEAN#SLOPE|
|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_MINI|   0.116845041513443|      MEAN|
|1F.EEK10|EKT72_PM1|LO_RF_VPP#STEP2_M...| 0.08191504329442978|      MEAN|
|1F.EEK10|EKT72_PM1|LO_RF_REF_POWER#A...| 0.06229864060878754|      MEAN|
|1F.EEK10|EKT72_PM1|PROCESS_GAS_8_O2#...| 0.06190823018550873|      MEAN|
|1F.EEK10|EKT72_PM1|LOWER_TEMPERATURE...|0.047050826251506805|      MEAN|
|1F.EEK10|EKT72_PM1|LO_C1_VAR_CAPACIT...|  0.0351719856262207|     SLOPE|
|1F.EEK10|EKT72_PM1|LO_C2_VAR_CAPACIT...| 0.03118673712015152|     RANGE|
|1F.EEK10|EKT72_PM1|EDGE_HE_FLOW#AOTU...|0.028021395206451416|      MEAN|
|1F.EEK10|EKT72_PM1|CENTER_GAS_PRESSU...| 0.02315637841820717|      MEAN|
|1F.EEK10|EKT72_PM1|CHAMBER_PRESSURE#.

In [ ]:
#####################################################################################
#############################将建模后的结果增加特定的列####################################
#####################################################################################
def add_certain_column(df, by, request_id):
    """
    param df: 最后的建模结果
    param by: 分组字段, 手动增加一列add
    param request_id: 传入的request_id
    return: 最后的建模结果增加特定的列
    """
    schema_all = StructType([ 
                             StructField("PRODG1", StringType(), True),
                              StructField("OPER_NO", StringType(), True),
                              StructField("TOOL_NAME", StringType(), True),
                              StructField("stats", StringType(), True),
                              StructField("parametric_name", StringType(), True),
                              StructField("weight", FloatType(), True),
                              StructField("request_id", StringType(), True),
                              StructField("weight_percent", FloatType(), True),
                              StructField("index_no", IntegerType(), True),
                              StructField("add", IntegerType(), True)]) 
    
    @pandas_udf(returnType=schema_all, functionType=PandasUDFType.GROUPED_MAP)
    def get_result(final_res):
        final_res['weight'] = final_res['weight'].astype(float)
        final_res = final_res.query("weight > 0")
        final_res['request_id'] = request_id
        final_res['weight_percent'] = final_res['weight']*100
        final_res = final_res.sort_values('weight', ascending=False)
        final_res['index_no'] = [i+1 for i in range(len(final_res))]
        # final_res['parametric_name'] = final_res['parametric_name'].str.replace("_", "+")
        return final_res
    return df.groupby(by).apply(get_result)

In [ ]:
#####################################################################################
################################将最后的结果写回数据库####################################
#####################################################################################
def doris_stream_load_from_df(df, engine, table, is_json=True, chunksize=100000, partitions=None):
    engine_url = engine.url
    url = 'http://%s:18030/api/%s/%s/_stream_load' % (engine_url.host, engine_url.database, table)

    format_str = 'csv' if not is_json else 'json'
    headers = {
        'Content-Type': 'text/plain; charset=UTF-8',
        'format': format_str,
        'Expect': '100-continue'
    }
    if is_json:
        headers['strip_outer_array'] = 'true'
        headers['read_json_by_line'] = 'true'
    else:
        headers['column_separator'] = '@'
    
    if partitions:
        headers['partitions'] = partitions
    
    auth = requests.auth.HTTPBasicAuth(engine_url.username, engine_url.password)
    session = requests.sessions.Session()
    session.should_strip_auth = lambda old_url, new_url: False
    
    l = len(df)
    if l > 0:
        if chunksize and chunksize < l:
            batches = l // chunksize
            if l % chunksize > 0:
                batches += 1
            for i in range(batches):
                si = i * chunksize
                ei = min(si + chunksize, l)
                sub = df[si:ei]
                do_doris_stream_load_from_df(sub, session, url, headers, auth, is_json)
        else:
            do_doris_stream_load_from_df(df, session, url, headers, auth, is_json)


def do_doris_stream_load_from_df(df, session, url, headers, auth, is_json=False):
    data = df.to_csv(header=False, index=False, sep='@') if not is_json else df.to_json(orient='records', date_format='iso')
    #print(data)
    
    resp = session.request(
        'PUT',
        url = url,
        data=data.encode('utf-8'),
        headers=headers,
        auth=auth
    )
    print(resp.reason, resp.text)
    check_stream_load_response(resp.text)


def check_stream_load_response(resp_text):
    resp = json.loads(resp_text)
    if resp['Status'] not in ["Success", "Publish Timeout"]:
        raise Exception(resp['Message'])



##########################################################################################
#######################################正式调用以上函数#######################################
##########################################################################################

# 1. 解析json 为字典， df1为kafka 输入的结果数据
df2 = df1.toPandas() 
# 默认只取第一行
if len(df2) > 0:
    df2 = df2.head(1)
request_id = df2["requestId"].values[0]
request_params = df2["requestParam"].values[0]
request_params = request_params.replace('\'', "\"")    # 避免存在单引号，因为json 引号只有双引号
parse_dict = json.loads(request_params)

# 2. 从kafka 关键字映射都具体数据源中的字段,没有的可以删除
keyword_map_from_json_to_table: dict = {
    "prodg1": "PRODG1",
    "waferId": "WAFER_ID",
    "dateRange": "START_TIME",
    "productId": "PRODUCT_ID",
    "operNo": "OPER_NO",
    "eqp": "EQP_NAME",
    "tool": "TOOL_NAME",
    "lot": "LOT_ID",
    "recipeName": "RECIPE_NAME"
}
# 获取查询条件list
select_condition_list = parse_dict

#  3. 查询表名, 需要修改
table_name = "etl.DWD_POC_CASE_FD_UVA_DATA_TEST"

# 4. 查询条件转sql,并读取数据
flag = True
try:
    select_df_list = [  read_sql(trans_select_condition_to_sql_with_label(select_condition_dict, table_name)) for select_condition_dict in select_condition_list  ]
    # 多个进行union
    df1 = reduce(DataFrame.unionAll, select_df_list)
except Exception as e:
    flag = False
    df_kafka = pd.DataFrame({"code": 1,  "msg": '系统内部异常! ',  'requestId': request_id}, index=[0])
    df_kafka_ = spark.createDataFrame(df_kafka)
    df1 = df_kafka_



# print(df1.count())
## 若能够正常读取数据，则df1不止一行, df1.count() > 1；若只有一行，则是df_kafka, df1.count() == 1
if flag == False:
    df1 = df1
else:
    # 1. 数据预处理
    df_run = _pre_process(df1)
    if df_run.count() == 0:
        df_kafka = pd.DataFrame({"code": 1,  "msg": '该条件下无数据，请检查！',  'requestId': request_id}, index=[0])
        df_kafka_ = spark.createDataFrame(df_kafka)
        df1 = df_kafka_
    else:
        # 2. 进行共性分析
        common_res = commonality_analysis(df_run)
        common_res.show()

        # 3. 挑选出数据：bad和good要同时大于3
        data_dict_bs = get_data_dict_big_sample(common_res)
        # print(data_dict_bs)
        if len(data_dict_bs) == 0:
            df_kafka = pd.DataFrame({"code": 1,  "msg": '以PRODG1+OPER_NO+TOOL_NAME分组, BAD_WAFER和GOOD_WAFER数量都要大于3片, 请检查数据! ',  'requestId': request_id}, index=[0])
            df_kafka_ = spark.createDataFrame(df_kafka)
            df1 = df_kafka_
        else:
            df_run_bs = get_train_data_big_sample(df_run, data_dict_bs)
            flag_al = True
            try:
                # 4. 对挑选出的数据进行建模
                res = fit_rf_big_sample(df=df_run_bs, by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
            except Exception as e:
                flag_al = False
                df_kafka = pd.DataFrame({"code": 1,  "msg": '算法内部异常! ',  'requestId': request_id}, index=[0])
                df_kafka_ = spark.createDataFrame(df_kafka)
                df1 = df_kafka_
            
            if flag_al == False:
                df1 = df1
            else:
                # 5. 获取所有bad wafer数量
                bad_wafer_num_big_sample = get_all_bad_wafer_num(df_run_bs)
                if bad_wafer_num_big_sample < 3:
                    df_kafka = pd.DataFrame({"code": 1,  "msg": 'BAD_WAFER数量需要大于3片, 请检查数据! ',  'requestId': request_id}, index=[0])
                    df_kafka_ = spark.createDataFrame(df_kafka)
                    df1 = df_kafka_
                else:
                    flag_score = True
                    try:
                        # 6. 将建模结果进行整合
                        s_res = split_score_big_sample(df=res, by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
                    except Exception as e:
                        flag_score = False
                        df_kafka = pd.DataFrame({"code": 1,  "msg": '算法运行评分结果较低，暂无输出! ',  'requestId': request_id}, index=[0])
                        df_kafka_ = spark.createDataFrame(df_kafka)
                        df1 = df_kafka_

                    if flag_score == False:
                        df1 = df1
                    else:
                        f_res = split_calculate_features_big_sample(df=res, by=['PRODG1', 'OPER_NO', 'TOOL_NAME'])
                        # s_res.show()
                        model_res_bs = get_finall_results_big_sample(s_res=s_res, f_res=f_res, bad_wafer_num=bad_wafer_num_big_sample)

                        # 7. 增加特定的列
                        final_res_bs = model_res_bs.withColumn('add', lit(0))
                        ddd = add_certain_column(df=final_res_bs, by='add', request_id=request_id)
                        ddd = ddd.drop('add')
                        ddd = ddd.toPandas()

                        # 8. ddd 是最后的结果，要写回数据库
                        user ="root"
                        host = "10.52.199.81"
                        password = "Nexchip%40123"
                        db = "etl"
                        port = 9030
                        engine = create_engine("mysql+pymysql://{user}:{password}@{host}:{port}/{db}".format(user = user,
                                                                                                            password = password,
                                                                                                            host = host,
                                                                                                            port = port,
                                                                                                            db = db))
                        doris_stream_load_from_df(ddd, engine, "results")



                        # 9. 最终成功的话，就会输出下面这条
                        df_kafka = pd.DataFrame({"code": 0, "msg": '运行成功', 'requestId': request_id}, index=[0])
                        df_kafka_ = spark.createDataFrame(df_kafka)
                        df1 = df_kafka_